<a href="https://colab.research.google.com/github/victor-lopes29/TCC/blob/main/TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import *
from nltk.corpus import stopwords
!pip install -U gensim
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import classification_report, make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import json
import time
import pickle
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
import os
from google.colab import drive

drive.mount('/content/drive')

path = "/content/drive/MyDrive/TCC/Dados"

files = os.listdir(path)

Mounted at /content/drive


In [ ]:
import os

file_path = "/content/drive/MyDrive/TCC/Dados/Doc2Vec/meu_doc2vec"

print(f"Tamanho do arquivo: {os.path.getsize(file_path)} bytes")

with open(file_path, "rb") as f:
    first_bytes = f.read(100)
    print("Primeiros bytes do arquivo:", first_bytes)


Tamanho do arquivo: 15667775 bytes
Primeiros bytes do arquivo: b'\x80\x04\x95\x00\x00\x01\x00\x00\x00\x00\x00\x8c\x15gensim.models.doc2vec\x94\x8c\x07Doc2Vec\x94\x93\x94)\x81\x94}\x94(\x8c\ndbow_words\x94K\x00\x8c\tdm_concat\x94K\x00\x8c\x0cdm_tag_count\x94K\x01\x8c'


# Carregando o Doc2Vec e testando

In [ ]:
fname = '/content/drive/MyDrive/TCC/Dados/Doc2Vec/meu_doc2vec'
model_dbow = Doc2Vec.load(fname)

In [ ]:
# Verificar o tamanho dos vetores
print(f"Tamanho dos vetores: {model_dbow.vector_size}")

# Verificar quantos documentos foram treinados
print(f"Quantidade de documentos no vocabulário: {len(model_dbow.dv)}")

# Listar algumas palavras presentes no vocabulário do modelo
print(f"Algumas palavras do vocabulário: {list(model_dbow.wv.index_to_key)[:10]}")


Tamanho dos vetores: 1000
Quantidade de documentos no vocabulário: 116910
Algumas palavras do vocabulário: [' ', 'a', 'e', 'o', 'r', 's', 'i', 't', 'n', 'd']


In [ ]:
# Pegar o vetor do primeiro documento
vector = model_dbow.dv[0]
print(f"Vetor do primeiro documento:\n{vector}")

Vetor do primeiro documento:
[ 1.67893499e-01 -9.59715769e-02  6.22485578e-02 -7.49798771e-03
  1.06030323e-01 -1.66047495e-02  2.95609199e-02  7.14868829e-02
 -8.19778964e-02 -6.30815998e-02  3.72685716e-02  7.09560066e-02
  6.89971447e-02  1.12894781e-01 -3.71208489e-02  7.88825452e-02
  1.93823241e-02 -4.32453491e-02  1.46849766e-01 -1.17435090e-01
  6.79206685e-04 -4.74756472e-02 -1.63118839e-01  5.59249111e-02
  2.65800990e-02  8.36029723e-02  8.49194452e-02 -1.11178465e-01
  7.66101256e-02  8.61720070e-02  1.89541832e-01  1.25876456e-01
  5.58292717e-02 -1.98832359e-02 -1.11116804e-01 -7.15977922e-02
 -1.50187612e-01  8.14781785e-02  1.08477749e-01  5.87881505e-02
 -2.04877257e-02 -1.39233395e-02 -3.65765728e-02 -7.72505477e-02
 -5.43245971e-02  1.43563747e-02  5.25061712e-02 -1.14970893e-01
  4.71415818e-02 -8.92853662e-02 -7.70967337e-04  2.55532353e-03
  3.83010507e-02 -3.83300148e-02 -1.31818131e-01  3.27825360e-02
  9.19200778e-02  4.86895852e-02  4.02680188e-02  1.35716408e

In [ ]:
# Encontrar os 5 documentos mais similares ao primeiro
similar_docs = model_dbow.dv.most_similar(0, topn=5)
print(f"Documentos mais similares ao primeiro: {similar_docs}")


Documentos mais similares ao primeiro: [(99464, 0.8115514516830444), (106603, 0.7995979189872742), (4513, 0.7983075976371765), (4787, 0.7942085862159729), (261, 0.7788448333740234)]


In [ ]:
nova_frase = "Esse jogo é muito bom"
vetor = model_dbow.infer_vector(nova_frase.split())  # Gera um vetor para a frase
print(model_dbow.dv.most_similar([vetor], topn=5))  # Encontra os documentos mais parecidos


[(105969, 0.9028264284133911), (105756, 0.8919234871864319), (105687, 0.8876928687095642), (25457, 0.8848954439163208), (104560, 0.8838923573493958)]


In [ ]:
print(model_dbow.dv.most_similar(0, topn=5))  # Mostra os 5 documentos mais similares ao primeiro


[(99464, 0.8115514516830444), (106603, 0.7995979189872742), (4513, 0.7983075976371765), (4787, 0.7942085862159729), (261, 0.7788448333740234)]


In [ ]:
print(list(model_dbow.dv.index_to_key)[:20])  # Mostra os primeiros 20 documentos armazenados

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [ ]:
print(model_dbow.dv.most_similar(0, topn=5))  # Mostra os 5 documentos mais similares ao primeiro


[(99464, 0.8115514516830444), (106603, 0.7995979189872742), (4513, 0.7983075976371765), (4787, 0.7942085862159729), (261, 0.7788448333740234)]


# Treinamento Word2Vec

In [ ]:
import os

# Caminho base onde estão os arquivos
reviews_path = "/content/drive/MyDrive/TCC/Dados/Reviews Da Steam filtradas/"

# Listar tudo dentro da pasta
conteudo = os.listdir(reviews_path)

# Separar arquivos e diretórios
pastas = [f for f in conteudo if os.path.isdir(os.path.join(reviews_path, f))]
arquivos = [f for f in conteudo if os.path.isfile(os.path.join(reviews_path, f))]

print("📂 Diretórios encontrados:", pastas)
print("📄 Arquivos encontrados:", arquivos)


📂 Diretórios encontrados: ['Action', 'Combined', 'Adventure', 'FPS', 'Horror', 'Sports', 'Simulation', 'Indie', 'RPG', 'Racing', 'Strategy']
📄 Arquivos encontrados: ['02_A_Word2Vec.ipynb', 'codigo_final.ipynb', 'concat.ipynb', '01_featurization-word2vec.ipynb', 'Racing_json_rest_part_50.json', 'Action_json_rest_part_50.json', 'Adventure_json_rest_part_50.json', 'FPS_json_rest_part_50.json', 'Horror_json_rest_part_50.json', 'Indie_json_rest_part_50.json', 'RPG_json_rest_part_50.json', 'Simulation_json_rest_part_50.json', 'Sports_json_rest_part_50.json', 'Strategy_json_rest_part_50.json', 'Combined_json_rest_part_50.json']


In [ ]:
import os
import json

# Caminho principal
reviews_path = "/content/drive/MyDrive/TCC/Dados/Reviews Da Steam filtradas/"

# Listar diretórios que representam os gêneros
generos = [f for f in os.listdir(reviews_path) if os.path.isdir(os.path.join(reviews_path, f))]

# Escolher um gênero para exemplo
genero_exemplo = generos[0]  # Pegando o primeiro gênero da lista
genero_path = os.path.join(reviews_path, genero_exemplo)

# Listar arquivos JSON dentro do diretório do gênero
arquivos_json = [f for f in os.listdir(genero_path) if f.endswith(".json")]

print(f"📂 Gênero escolhido: {genero_exemplo}")
print(f"📄 Arquivos JSON encontrados: {arquivos_json}")

# Exemplo: Carregar um JSON
if arquivos_json:
    json_path = os.path.join(genero_path, arquivos_json[0])  # Pegando o primeiro JSON
    with open(json_path, "r", encoding="utf-8") as f:
        dados = json.load(f)
        print(f"🔍 Exemplo de dados: {list(dados.items())[:2]}") # Exibir apenas os 2 primeiros registros


📂 Gênero escolhido: Action
📄 Arquivos JSON encontrados: ['Action_SkatemastaTcheco.json', 'Action_SteelRats.json', 'Action_ATOMEGA.json', 'Action_JustCause3MultiplayerMod.json', 'Action_VroomKaboom.json', 'Action_MiniBattlegrounds.json', 'Action_SamuraiRiot.json', 'Action_ViciousAttackLlamaApocalypse.json', 'Action_OffensiveCombatRedux.json', 'Action_SteampunkTower2.json', 'Action_MayheminSingleValley.json', 'Action_Hacktag.json', 'Action_Dungeons&DragonsDarkAlliance.json', 'Action_BloodAncestors.json', 'Action_AcanaHeart3LOVEMAXSIXSTARSXTEND.json', 'Action_AssassinsCreedOriginsTheHiddenOnes.json', 'Action_review_624910.json', 'Action_MysteriaOccultShadows.json', 'Action_HammerHelm.json', 'Action_Zombotron.json', 'Action_DungeonsofTalDoria.json', 'Action_TheInitial.json', 'Action_Injustice2.json', 'Action_IdleChampionsOfTheFOrgottenRealms.json', 'Action_DrunkenWrestlers2.json', 'Action_SkyForceReloaded.json', 'Action_DeadMaze.json', 'Action_CryptStalker.json', 'Action_Espire1VROperative

In [ ]:
# Extrair apenas os reviews corretamente
reviews = dados['reviews']  # Acessa o dicionário de reviews
exemplo_reviews = list(reviews.values())[:2]  # Pega os dois primeiros comentários
print(f"🔍 Exemplo de dados: {exemplo_reviews}")


🔍 Exemplo de dados: [{'recommendationid': '106208136', 'author': {'steamid': '76561198278381555', 'num_games_owned': 218, 'num_reviews': 27, 'playtime_forever': 397, 'playtime_last_two_weeks': 0, 'playtime_at_review': 397, 'last_played': 1625863019}, 'language': 'brazilian', 'review': 'Fodda.', 'timestamp_created': 1640050749, 'timestamp_updated': 1640050749, 'voted_up': True, 'votes_up': 1, 'votes_funny': 0, 'weighted_vote_score': '0.500652730464935303', 'comment_count': 0, 'steam_purchase': False, 'received_for_free': False, 'written_during_early_access': False}, {'recommendationid': '104230164', 'author': {'steamid': '76561198424678766', 'num_games_owned': 73, 'num_reviews': 14, 'playtime_forever': 28, 'playtime_last_two_weeks': 0, 'playtime_at_review': 20, 'last_played': 1639332609}, 'language': 'brazilian', 'review': 'agora eu sei como andar de skate', 'timestamp_created': 1637972262, 'timestamp_updated': 1637972262, 'voted_up': True, 'votes_up': 1, 'votes_funny': 0, 'weighted_vot

In [ ]:
# Extrair apenas os textos dos comentários
comentarios = [review["review"] for review in reviews.values()]

# Exibir alguns exemplos
print(f"🔍 Exemplo de comentários: {comentarios[:5]}")

🔍 Exemplo de comentários: ['Fodda.', 'agora eu sei como andar de skate', 'jogo pica tipo tony hawk, só que melho \n⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⢀⡀⣼⣵⣽⣿⠄⡂⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄ ⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⣴⣷⣿⣿⣿⣿⣷⣷⣷⣴⢠⣄⠄⢠⠄⠄⠄⠄⠄ ⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⣬⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣾⣷⣄⠄⠄⠄ ⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿⠃⠄⠄⠄ ⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⠛⠛⠛⠛⠛⠛⠿⣿⣿⣿⣿⢿⠛⣉⣥⣤⣀⠄⠄⠄ ⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⢀⣤⡴⠶⠶⠶⢶⣿⣿⣿⣿⠛⢛⠛⠛⡉⠛⠄⠄⠄ ⠄⠄⠄⡀⠄⠄⠄⠄⠄⣴⣿⣡⣤⣤⣥⣿⣤⣢⠉⣿⣿⠄⣶⣶⣶⣶⣦⣦⡄⠄ ⠄⠄⠄⢋⣀⠄⠄⠄⠄⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣿⣿⠄⣿⣿⣿⣿⣿⣿⡏⠄ ⠄⠄⠄⠈⢿⣆⠄⠄⠄⢻⣿⣿⣿⣿⣿⣿⣿⠿⠛⣿⣿⣦⡘⣿⣿⣿⣿⣿⠃⠄ ⠄⠄⠄⠄⠈⢻⣷⠄⠄⠈⣿⣿⣿⣿⣿⣿⣿⢼⣿⠿⣿⣿⣟⣽⣿⣿⣿⡟⠄⠄ ⣷⡄⠄⠄⠄⠄⠉⠄⠄⠄⣶⣿⣿⣿⣿⣿⣿⣿⣾⣿⣶⣿⣿⣿⣿⣿⣿⡇⡄⠄ ⠛⠁⠄⠄⠄⠄⠄⠄⠄⠄⠻⣿⣿⣿⣿⡛⠛⢛⣛⣟⣟⣻⣯⠍⢩⣿⣿⡏⠄⠄ ⠄⠄⠄⢠⢠⠄⠂⠄⠄⠄⠄⠙⢿⣿⣿⣿⣧⣌⣨⣯⣿⣯⠄⣤⣿⣿⡟⠃⠄⠄ ⠄⠄⠄⠋⠘⠄⠄⠄⠄⠄⠄⠄⠄⠻⣿⣿⣿⣿⣿⣿⣛⣿⣿⣿⣿⡟⠄⠄⠄⠄ ⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⠠⠄⠄⠄⠈⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠄⠄⠄⠄⠄', 'muito mais fodastico que o primeiro', 'Jogoé um pouco difícil, mas é engraçado. Bingo clandestino é de lascar']


Pré-processamento

In [ ]:
from nltk.stem import RSLPStemmer
import string
nltk.download('punkt_tab')

stopwords_pt = set(stopwords.words('portuguese'))
stemmer = RSLPStemmer()

def preprocess_text(text):
    """Aplica pré-processamento no texto: limpeza, remoção de stopwords e stemming."""
    # Remover pontuação e números
    text = ''.join([char.lower() for char in text if char not in string.punctuation and not char.isdigit()])

    # Tokenizar texto
    tokens = nltk.word_tokenize(text, language='portuguese')

    # Remover stopwords e aplicar stemming
    tokens = [stemmer.stem(word) for word in tokens if word not in stopwords_pt]

    return ' '.join(tokens)

# Caminho dos arquivos
caminho_dados = "/content/drive/MyDrive/TCC/Dados/Reviews Da Steam filtradas/Action"

# Lista para armazenar os comentários pré-processados
comentarios_processados = []

# Processar cada arquivo JSON dentro do diretório escolhido
for arquivo in os.listdir(caminho_dados):
    if arquivo.endswith(".json"):
        with open(os.path.join(caminho_dados, arquivo), "r", encoding="utf-8") as f:
            dados = json.load(f)

            # Extraindo apenas os comentários
            for review in dados.get('reviews', {}).values():
                texto_processado = preprocess_text(review.get('review', ''))
                comentarios_processados.append(texto_processado)

# Exibir exemplos dos comentários processados
print("🔍 Exemplos de comentários processados:")
print(comentarios_processados[:5])

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


🔍 Exemplos de comentários processados:
['fodd', 'agor sei and skat', 'jog pic tip tony hawk melh ⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⢀⡀⣼⣵⣽⣿⠄⡂⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄ ⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⣴⣷⣿⣿⣿⣿⣷⣷⣷⣴⢠⣄⠄⢠⠄⠄⠄⠄⠄ ⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⣬⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣾⣷⣄⠄⠄⠄ ⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿⠃⠄⠄⠄ ⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⠛⠛⠛⠛⠛⠛⠿⣿⣿⣿⣿⢿⠛⣉⣥⣤⣀⠄⠄⠄ ⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⢀⣤⡴⠶⠶⠶⢶⣿⣿⣿⣿⠛⢛⠛⠛⡉⠛⠄⠄⠄ ⠄⠄⠄⡀⠄⠄⠄⠄⠄⣴⣿⣡⣤⣤⣥⣿⣤⣢⠉⣿⣿⠄⣶⣶⣶⣶⣦⣦⡄⠄ ⠄⠄⠄⢋⣀⠄⠄⠄⠄⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣿⣿⠄⣿⣿⣿⣿⣿⣿⡏⠄ ⠄⠄⠄⠈⢿⣆⠄⠄⠄⢻⣿⣿⣿⣿⣿⣿⣿⠿⠛⣿⣿⣦⡘⣿⣿⣿⣿⣿⠃⠄ ⠄⠄⠄⠄⠈⢻⣷⠄⠄⠈⣿⣿⣿⣿⣿⣿⣿⢼⣿⠿⣿⣿⣟⣽⣿⣿⣿⡟⠄⠄ ⣷⡄⠄⠄⠄⠄⠉⠄⠄⠄⣶⣿⣿⣿⣿⣿⣿⣿⣾⣿⣶⣿⣿⣿⣿⣿⣿⡇⡄⠄ ⠛⠁⠄⠄⠄⠄⠄⠄⠄⠄⠻⣿⣿⣿⣿⡛⠛⢛⣛⣟⣟⣻⣯⠍⢩⣿⣿⡏⠄⠄ ⠄⠄⠄⢠⢠⠄⠂⠄⠄⠄⠄⠙⢿⣿⣿⣿⣧⣌⣨⣯⣿⣯⠄⣤⣿⣿⡟⠃⠄⠄ ⠄⠄⠄⠋⠘⠄⠄⠄⠄⠄⠄⠄⠄⠻⣿⣿⣿⣿⣿⣿⣛⣿⣿⣿⣿⡟⠄⠄⠄⠄ ⠄⠄⠄⠄⠄⠄⠄⠄⠄⠄⠠⠄⠄⠄⠈⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠄⠄⠄⠄⠄', 'fodas prim', 'jogoé pouc difícil engraç bing clandestin lasc']


In [ ]:
import re
import unicodedata

# Baixar pacotes do NLTK (caso ainda não tenha baixado)
nltk.download('punkt')
nltk.download('stopwords')

# Definir stopwords em português
stopwords_pt = set(stopwords.words('portuguese'))
stemmer = RSLPStemmer()

def limpar_texto(texto):
    # Normalizar para remover acentos
    texto = unicodedata.normalize("NFKD", texto).encode("ascii", "ignore").decode("utf-8", "ignore")

    # Remover arte ASCII e caracteres não alfabéticos (exceto espaços)
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)

    # Tokenizar o texto
    tokens = word_tokenize(texto.lower())

    # Remover stopwords e aplicar stemming
    tokens = [stemmer.stem(word) for word in tokens if word not in stopwords_pt]

    # Reunir tokens em uma string novamente
    return " ".join(tokens)

# Exemplo de aplicação no dataset
comentarios_processados = [limpar_texto(comentario) for comentario in comentarios]

# Visualizar os primeiros comentários processados
print("🔍 Exemplos de comentários processados:")
print(comentarios_processados[:5])


🔍 Exemplos de comentários processados:
['fodd', 'agor sei and skat', 'jog pic tip tony hawk so melh', 'fodas prim', 'jogo pouc dificil engrac bing clandestin lasc']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.stem import WordNetLemmatizer

# Baixar pacotes necessários do NLTK
nltk.download('wordnet')
nltk.download('omw-1.4')

# Definir stopwords em português
stopwords_pt = set(stopwords.words('portuguese'))
lemmatizer = WordNetLemmatizer()

def limpar_texto(texto):
    # Normalizar para remover acentos
    texto = unicodedata.normalize("NFKD", texto).encode("ascii", "ignore").decode("utf-8", "ignore")

    # Remover caracteres especiais e manter apenas letras e espaços
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)

    # Tokenizar o texto
    tokens = word_tokenize(texto.lower())

    # Remover stopwords e aplicar lematização (em vez de stemming)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords_pt and len(word) > 2]

    # Reunir tokens em uma string novamente
    return " ".join(tokens)

# Aplicar nos comentários
comentarios_processados = [limpar_texto(comentario) for comentario in comentarios]

# Visualizar os primeiros comentários processados
print("🔍 Exemplos de comentários processados:")
print(comentarios_processados[:5])


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


🔍 Exemplos de comentários processados:
['fodda', 'agora sei andar skate', 'jogo pica tipo tony hawk melho', 'fodastico primeiro', 'jogoe pouco dificil engracado bingo clandestino lascar']


In [ ]:
def limpar_texto(texto):
    # Normalizar para remover acentos
    texto = unicodedata.normalize("NFKD", texto).encode("ascii", "ignore").decode("utf-8", "ignore")

    # Remover caracteres especiais e manter apenas letras e espaços
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)

    # Tokenizar o texto
    tokens = word_tokenize(texto.lower())

    # Remover stopwords e aplicar lematização
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords_pt and len(word) > 2]

    # Remover palavras repetidas grudadas (ex: "jogoe" → "jogo")
    tokens = [word for word in tokens if not any(c.isdigit() for c in word)]  # Remove palavras com números (se houver)

    # Reunir tokens em uma string novamente e garantir espaçamento correto
    return " ".join(tokens).strip()

# Aplicar nos comentários
comentarios_processados = [limpar_texto(comentario) for comentario in comentarios]

# Visualizar os primeiros comentários processados
print("🔍 Exemplos de comentários processados:")
print(comentarios_processados[:5])


🔍 Exemplos de comentários processados:
['fodda', 'agora sei andar skate', 'jogo pica tipo tony hawk melho', 'fodastico primeiro', 'jogoe pouco dificil engracado bingo clandestino lascar']


In [ ]:
path_json = "/content/drive/MyDrive/TCC/Dados/Reviews Da Steam filtradas/Action/Action_SkatemastaTcheco.json"

# Carregar JSON
import json

with open(path_json, "r", encoding="utf-8") as f:
    dados = json.load(f)

# Extrair apenas os comentários
comentarios = [entry["review"] for entry in list(dados["reviews"].values())]

# Função para pré-processamento melhorado
def preprocess_text(texto):
    # Normalizar para remover acentos
    texto = unicodedata.normalize("NFKD", texto).encode("ascii", "ignore").decode("utf-8")

    # Tokenizar
    tokens = word_tokenize(texto.lower())

    # Remover stopwords
    stop_words = set(nltk.corpus.stopwords.words("portuguese"))
    tokens = [t for t in tokens if t not in stop_words]

    # Remover caracteres especiais e palavras muito curtas (1 ou 2 letras)
    tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens]
    tokens = [token for token in tokens if len(token) > 2]

    # Reunir tokens em texto novamente
    return " ".join(tokens).strip()

# Aplicar a função nos comentários
comentarios_processados = [preprocess_text(comentario) for comentario in comentarios]

# Exibir alguns exemplos processados
print("Exemplos de comentários processados:")
print(comentarios_processados[:5])

Exemplos de comentários processados:
['fodda', 'agora sei andar skate', 'jogo pica tipo tony hawk melho', 'fodastico primeiro', 'jogoe pouco dificil engracado bingo clandestino lascar']


In [ ]:
import pandas as pd
import unicodedata
import nltk
import re
nltk.download("punkt")
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Caminho do arquivo JSON
path_json = "/content/drive/MyDrive/TCC/Dados/Reviews Da Steam filtradas/Action/Action_SkatemastaTcheco.json"

# Carregar JSON
import json

with open(path_json, "r", encoding="utf-8") as f:
    dados = json.load(f)

# Extrair os comentários do JSON
comentarios = [entry["review"] for entry in dados["reviews"].values()]

# Função para pré-processamento de texto
def preprocess_text(texto):
    # Remover acentos
    texto = unicodedata.normalize("NFKD", texto).encode("ascii", "ignore").decode("utf-8")

    # Transformar para minúsculas
    texto = texto.lower()

    # Tokenizar
    tokens = word_tokenize(texto)

    # Remover stopwords e palavras muito curtas
    stop_words = set(nltk.corpus.stopwords.words("portuguese"))
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]

    # Remover caracteres não alfabéticos isolados (como pontuações perdidas na tokenização)
    tokens = [re.sub(r"[^a-z]", "", token) for token in tokens]
    tokens = [token for token in tokens if token]  # Remove strings vazias geradas pela regex

    return " ".join(tokens)  # Reunir os tokens processados em um texto novamente

# Aplicar o pré-processamento aos comentários
comentarios_processados = [preprocess_text(comentario) for comentario in comentarios]

# Exibir alguns exemplos processados
print("Exemplos de comentários processados:")
print(list(enumerate(comentarios_processados))[:5])


Exemplos de comentários processados:
[(0, 'fodda'), (1, 'agora sei andar skate'), (2, 'jogo pica tipo tony hawk melho'), (3, 'fodastico primeiro'), (4, 'jogoe pouco dificil engracado bingo clandestino lascar')]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from gensim.models import Word2Vec

diretorio = "/content/drive/MyDrive/TCC/Dados/Word2Vec"

# Converter os comentários processados em listas de tokens
comentarios_tokens = [comentario.split() for comentario in comentarios_processados]

# Treinar o modelo Word2Vec
modelo_w2v = Word2Vec(sentences=comentarios_tokens, vector_size=100, window=5, min_count=1, workers=4)

# Caminho completo para salvar o modelo
caminho_modelo = os.path.join(diretorio, "word2vec_modelo.model")

# Salvar o modelo Word2Vec
modelo_w2v.save(caminho_modelo)

# Exemplo: Obter a similaridade entre palavras
similaridade = modelo_w2v.wv.similarity('skate', 'tony')
print(f"Similaridade entre 'skate' e 'tony': {similaridade}")


Similaridade entre 'skate' e 'tony': 0.1346835494041443


In [ ]:
import os
import json

# Caminho da pasta com os JSONs
diretorio_json = "/content/drive/MyDrive/TCC/Dados/Reviews Da Steam filtradas/Combined/part50/"

# Percorrer todos os arquivos JSON e verificar a estrutura
for arquivo in os.listdir(diretorio_json):
    if arquivo.endswith(".json"):
        caminho_arquivo = os.path.join(diretorio_json, arquivo)

        with open(caminho_arquivo, "r", encoding="utf-8") as f:
            dados = json.load(f)

        print(f"\n📂 Estrutura do arquivo: {arquivo}")
        print(dados.keys())  # Mostra as chaves principais do JSON

        print(type(dados))  # Isso nos dirá se é uma lista ou um dicionário
        print(dados.keys() if isinstance(dados, dict) else dados[:3])



📂 Estrutura do arquivo: Racing_json_part50.json
dict_keys(['recommendationid', 'language', 'review', 'timestamp_created', 'timestamp_updated', 'voted_up', 'votes_up', 'votes_funny', 'weighted_vote_score', 'comment_count', 'steam_purchase', 'received_for_free', 'written_during_early_access', 'author.steamid', 'author.num_games_owned', 'author.num_reviews', 'author.playtime_forever', 'author.playtime_last_two_weeks', 'author.playtime_at_review', 'author.last_played', 'timestamp_dev_responded', 'developer_response'])
<class 'dict'>
dict_keys(['recommendationid', 'language', 'review', 'timestamp_created', 'timestamp_updated', 'voted_up', 'votes_up', 'votes_funny', 'weighted_vote_score', 'comment_count', 'steam_purchase', 'received_for_free', 'written_during_early_access', 'author.steamid', 'author.num_games_owned', 'author.num_reviews', 'author.playtime_forever', 'author.playtime_last_two_weeks', 'author.playtime_at_review', 'author.last_played', 'timestamp_dev_responded', 'developer_resp

In [ ]:
from tqdm import tqdm
stop_words = set(stopwords.words('portuguese'))

# Pasta onde os arquivos JSON estão armazenados
pasta_dados = "/content/drive/MyDrive/TCC/Dados/Reviews Da Steam filtradas/Combined/part50/"  # Modifique para o caminho correto

# Função para carregar os dados e extrair comentários
def carregar_comentarios(pasta):
    comentarios = []
    for arquivo in tqdm(os.listdir(pasta), desc="Lendo arquivos"):
        caminho = os.path.join(pasta, arquivo)

        # Ler arquivo JSON
        with open(caminho, "r", encoding="utf-8") as f:
            dados = json.load(f)  # O JSON é um dicionário

            # Verificar se a chave "review" existe e armazenar seus valores
            if "review" in dados:
                comentarios.extend(dados["review"].values())  # Pegar todos os comentários

    return comentarios

# Testando a função
comentarios = carregar_comentarios(pasta_dados)

print(f"Total de comentários carregados: {len(comentarios)}")
print("Exemplo de comentário:", comentarios[:3])  # Exibe os 3 primeiros comentários

# Função para pré-processar os textos
def preprocessar_texto(texto):
    texto = texto.lower()  # Converter para minúsculas
    texto = re.sub(r'[^\w\s]', '', texto)  # Remover pontuação
    texto = re.sub(r'[^a-záéíóúãõâêôç\s]', '', texto)  # Remover caracteres estranhos
    tokens = word_tokenize(texto)  # Tokenizar
    tokens = [palavra for palavra in tokens if palavra not in stop_words and len(palavra) > 2]  # Remover stopwords e palavras curtas
    return tokens

# Carregar e processar os comentários
comentarios = carregar_comentarios(pasta_dados)
comentarios_tokenizados = [preprocessar_texto(comentario) for comentario in tqdm(comentarios, desc="Processando comentários")]

# Treinar modelo Word2Vec
modelo_w2v = Word2Vec(sentences=comentarios_tokenizados, vector_size=1000, window=5, min_count=5, workers=4)

# Salvar o modelo treinado
modelo_w2v.save(caminho_modelo)
print("Modelo Word2Vec salvo com sucesso!")

Lendo arquivos: 100%|██████████| 10/10 [00:01<00:00,  7.21it/s]


Total de comentários carregados: 116910
Exemplo de comentário: ['Pior coisa foi a EA ter comprado, o  F1-2020  nem de longe deu tanto problema quanto esse, simplesmente impossível de jogar com esse BUG  que faz o jogo dar crash, e o pior nem se pronunciam com uma previsão de correção. \nEstava empolgado para jogar a versão nova e as pistas mas simplesmente não tem como no máximo consigo ficar 5 6 voltas em time trial.\nNa boa enquanto não corrigirem esse problema guarde o seu dinheiro e evite passar raiva.\n\n', '- O jogo tem o foco mais em sobrevivência na pista, do que a própria corrida em si.\n\n- O estilo de música e competição no jogo é um pouco infantil, mas não deixa de ser legal de jogar com outras pessoas ou com o próprio computador. Pois acaba sendo divertido.\n\n- Cenário bem otimizado. \n\n- As vezes o computador joga bem melhor que um adulto. Não é tão fácil assim rs\n\n', 'Como todo simulador, indico jogar com volante/pedal pra não passar raiva.\n\nA sensação de velocidad

Processando comentários: 100%|██████████| 116910/116910 [00:43<00:00, 2708.89it/s]


Modelo Word2Vec salvo com sucesso!


# Testando o Word2Ved

In [ ]:
# Carregar o modelo treinado
fword2vec = '/content/drive/MyDrive/TCC/Dados/Word2Vec/word2vec_modelo.model'
modelo_w2v = Word2Vec.load(fword2vec)

# Testar palavras similares
#print(modelo_w2v.wv.most_similar("jogo", topn=5))
#print(modelo_w2v.wv.most_similar("gráficos", topn=5))
#print(modelo_w2v.wv.most_similar("história", topn=5))

print(f"Tamanho do vocabulário: {len(modelo_w2v.wv)}")

Tamanho do vocabulário: 41297


# Transformar o Word2Vec em Embeddings

In [ ]:
pasta_dados = "/content/drive/MyDrive/TCC/Dados/Reviews Da Steam filtradas/Combined/part50/"  # Modifique para o caminho correto

def carregar_comentarios(pasta):
    comentarios = []
    for arquivo in tqdm(os.listdir(pasta), desc="Lendo arquivos"):
        caminho = os.path.join(pasta, arquivo)

        # Ler arquivo JSON
        with open(caminho, "r", encoding="utf-8") as f:
            dados = json.load(f)  # O JSON é um dicionário

            # Verificar se a chave "review" existe e armazenar seus valores
            if "review" in dados:
                comentarios.extend(dados["review"].values())  # Pegar todos os comentários

    return comentarios

comentarios = carregar_comentarios(pasta_dados)

Lendo arquivos: 100%|██████████| 10/10 [00:14<00:00,  1.43s/it]


In [ ]:
def vectorizar_comentario(comentario, modelo):
    tokens = comentario.split()  # Tokenizar o texto
    vetores = [modelo.wv[palavra] for palavra in tokens if palavra in modelo.wv]  # Buscar embeddings válidos
    return np.mean(vetores, axis=0) if vetores else np.zeros(modelo.vector_size)  # Média dos vetores

# Criar matriz de vetores para os comentários
X_word2vec = np.array([vectorizar_comentario(c, modelo_w2v) for c in comentarios])

#print(f"Formato de X_word2vec: {X_word2vec.shape}")  # Deve ser algo como (n_comentarios, 100)


NameError: name 'modelo_w2v' is not defined

# Concatenar o Doc2Vec com Word2Vec

In [ ]:
# Caminho do arquivo com os vetores Doc2Vec
caminho_doc2vec = "/content/drive/MyDrive/TCC/Dados/Doc2Vec/meu_doc2vec.dv.vectors.npy"

# Carregar os embeddings Doc2Vec
X_doc2vec = np.load(caminho_doc2vec)

X_combinado = np.hstack((X_word2vec, X_doc2vec))  # Une horizontalmente

# Caminho para salvar no Google Drive
caminho_concatenacao = "/content/drive/MyDrive/TCC/Dados/X_combinado.npy"

# Salvar os embeddings combinados
np.save(caminho_concatenacao, X_combinado)

print(f"Formato final dos embeddings combinados: {X_combinado.shape}")  # Deve ser (n_amostras, 2000)


Formato final dos embeddings combinados: (116910, 2000)


In [ ]:
# Caminho do arquivo salvo no Google Drive
caminho_concatenacao = "/content/drive/MyDrive/TCC/Dados/X_combinado.npy"

# Carregar os embeddings combinados
X_combinado = np.load(caminho_concatenacao)

In [ ]:
# Definir o número de amostras desejado
n_amostras = 60000

# Verificar se temos pelo menos 60.000 exemplos
if len(comentarios) < n_amostras:
    raise ValueError(f"O conjunto de dados tem apenas {len(comentarios)} exemplos, impossível selecionar {n_amostras}.")

# Selecionar 60.000 comentários mantendo a distribuição de classes
comentarios_selecionados, _, y_selecionado, _ = train_test_split(comentarios, y, train_size=n_amostras, stratify=y, random_state=42)

print(f"Novo número de comentários: {len(comentarios_selecionados)}")
print(f"Número de rótulos selecionados: {len(y_selecionado)}")


Novo número de comentários: 60000
Número de rótulos selecionados: 60000


In [ ]:
fword2vec = '/content/drive/MyDrive/TCC/Dados/Word2Vec/word2vec_modelo.model'
modelo_w2v = Word2Vec.load(fword2vec)
X_word2vec = np.array([vectorizar_comentario(c, modelo_w2v) for c in comentarios_selecionados])
print(f"Formato de X_word2vec: {X_word2vec.shape}")  # Deve ser (60000, 1000)

Formato de X_word2vec: (60000, 1000)


In [ ]:
# Carregar os embeddings completos do Doc2Vec
X_doc2vec_total = np.load("/content/drive/MyDrive/TCC/Dados/Doc2Vec/meu_doc2vec.dv.vectors.npy")

# Selecionar apenas os 60.000 que correspondem aos comentários escolhidos
X_doc2vec = X_doc2vec_total[:60000]

print(f"Formato de X_doc2vec: {X_doc2vec.shape}")  # Deve ser (60000, 1000)

Formato de X_doc2vec: (60000, 1000)


In [ ]:
# Concatenar Word2Vec e Doc2Vec
X_combinado = np.hstack((X_word2vec, X_doc2vec))

# Caminho para salvar no Google Drive
caminho_concatenacao_reduzida = "/content/drive/MyDrive/TCC/Dados/X_combinado_reduzido.npy"

# Salvar os embeddings combinados
np.save(caminho_concatenacao_reduzida, X_combinado)

print(f"Novo formato dos embeddings combinados: {X_combinado.shape}")  # Deve ser (60000, 2000)

Novo formato dos embeddings combinados: (60000, 2000)


# Separar modelos de Treinos e de Teste (60.000)

Gerar o Y usando o "voted_up"

In [ ]:
# Lista para armazenar os rótulos
y = []

# Pasta onde estão os arquivos JSON
pasta_dados = "/content/drive/MyDrive/TCC/Dados/Reviews Da Steam filtradas/Combined/part50/"

# Percorrer todos os arquivos JSON
for arquivo in tqdm(os.listdir(pasta_dados), desc="Lendo arquivos"):
    caminho = os.path.join(pasta_dados, arquivo)

    with open(caminho, "r", encoding="utf-8") as f:
        dados = json.load(f)  # Carrega o arquivo JSON

        # Coletar o valor de "voted_up" e converter True → 1, False → 0
        if "voted_up" in dados:
            y.extend([int(valor) for valor in dados["voted_up"].values()])  # Converte booleanos para inteiros

# Converter para array NumPy
y = np.array(y)

print(f"Total de rótulos carregados: {len(y)}")
print(f"Exemplo de rótulos: {y[:10]}")  # Mostrar alguns rótulos

Lendo arquivos: 100%|██████████| 10/10 [00:04<00:00,  2.04it/s]

Total de rótulos carregados: 116910
Exemplo de rótulos: [0 1 1 1 1 1 1 0 1 1]


Dados Reduzidos

In [ ]:
# Caminho do arquivo salvo no Google Drive
caminho_concatenacao_reduzida = "/content/drive/MyDrive/TCC/Dados/X_combinado_reduzido.npy"

# Carregar os embeddings combinados
X_combinado = np.load(caminho_concatenacao_reduzida)

KeyboardInterrupt: 

Separar os dados

In [ ]:
y_selecionado = y[:60000]

# Dividir os dados com 50% treino e 50% teste
X_train, X_test, y_train, y_test = train_test_split(X_combinado, y_selecionado, test_size=0.5, random_state=42)

print(f"Tamanho do conjunto de treino: {len(X_train)} comentários")
print(f"Tamanho do conjunto de teste: {len(X_test)} comentários")

NameError: name 'X_combinado' is not defined

Dados Completos

In [18]:
# Caminho do arquivo salvo no Google Drive
caminho_concatenacao = "/content/drive/MyDrive/TCC/Dados/X_combinado.npy"

# Carregar os embeddings combinados
X_combinado = np.load(caminho_concatenacao)

Separar os dados completos

In [13]:
# Dividir os dados com 50% treino e 50% teste
X_train, X_test = train_test_split(X_combinado, test_size=0.5, random_state=42)

print(f"Tamanho do conjunto de treino: {len(X_train)} comentários")
print(f"Tamanho do conjunto de teste: {len(X_test)} comentários")

Tamanho do conjunto de treino: 58455 comentários
Tamanho do conjunto de teste: 58455 comentários


# Definir a validação cruzada

In [ ]:
# Definir os hiperparâmetros a serem testados
param_grid_svm = {
    'kernel': ['linear', 'rbf', 'poly'],  # Testar diferentes tipos de kernel
    'C': [0.1, 1, 10]  # Testar diferentes penalizações
}

# Configurar validação cruzada estratificada para manter equilíbrio de classes
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Criar GridSearchCV com verbose para mostrar progresso
grid_svm = GridSearchCV(SVC(), param_grid_svm, cv=cv, scoring='f1', n_jobs=-1, verbose=3)

# Mensagem de início
print("🔄 Iniciando GridSearchCV para SVM...")
start_time = time.time()

# Rodar o GridSearchCV (com logs de progresso)
grid_svm.fit(X_combinado, y_selecionado)

# Mensagem de fim
end_time = time.time()
tempo_total = end_time - start_time
print(f"✅ GridSearchCV finalizado em {tempo_total/60:.2f} minutos.")

# Melhor configuração encontrada
print("🎯 Melhores parâmetros para SVM:", grid_svm.best_params_)

🔄 Iniciando GridSearchCV para SVM...
Fitting 5 folds for each of 9 candidates, totalling 45 fits


KeyboardInterrupt: 

In [ ]:
# Definir os hiperparâmetros a serem testados
param_grid_svm = {
    'kernel': ['linear'],  # Apenas linear para acelerar
    'C': [0.1, 1, 10]
}

# Configurar validação cruzada estratificada para manter equilíbrio de classes
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Criar GridSearchCV com verbose para mostrar progresso
grid_svm = GridSearchCV(SVC(), param_grid_svm, cv=cv, scoring='f1', n_jobs=-1, verbose=3)

# Mensagem de início
print("🔄 Iniciando GridSearchCV para SVM...")
start_time = time.time()

# Rodar o GridSearchCV (com logs de progresso)
grid_svm.fit(X_combinado, y_selecionado)

# Mensagem de fim
end_time = time.time()
tempo_total = end_time - start_time
print(f"✅ GridSearchCV finalizado em {tempo_total/60:.2f} minutos.")

# Melhor configuração encontrada
print("🎯 Melhores parâmetros para SVM:", grid_svm.best_params_)

🔄 Iniciando GridSearchCV para SVM...
Fitting 3 folds for each of 3 candidates, totalling 9 fits


In [ ]:
# Definir os hiperparâmetros a serem testados
param_grid_svm = {
    'kernel': ['linear'],  # Apenas linear para acelerar
    'C': [0.1, 1, 10]
}

# Configurar validação cruzada estratificada para manter equilíbrio de classes
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Criar GridSearchCV com verbose para mostrar progresso
grid_svm = GridSearchCV(SVC(), param_grid_svm, cv=cv, scoring='f1', n_jobs=-1, verbose=3)

# Reduzir para 20.000 amostras para um teste rápido
X_teste, _, y_teste, _ = train_test_split(X_combinado, y_selecionado, train_size=20000, stratify=y_selecionado, random_state=42)

# Mensagem de início
print("🔄 Iniciando GridSearchCV para SVM...")
start_time = time.time()

# Rodar o GridSearch apenas nesse subconjunto
grid_svm.fit(X_teste, y_teste)

# Mensagem de fim
end_time = time.time()
tempo_total = end_time - start_time
print(f"✅ GridSearchCV finalizado em {tempo_total/60:.2f} minutos.")

# Melhor configuração encontrada
print("🎯 Melhores parâmetros para SVM:", grid_svm.best_params_)

🔄 Iniciando GridSearchCV para SVM...
Fitting 3 folds for each of 3 candidates, totalling 9 fits
✅ GridSearchCV finalizado em 410.74 minutos.
🎯 Melhores parâmetros para SVM: {'C': 0.1, 'kernel': 'linear'}


# Treino do SVM

In [ ]:
# Criar o modelo LinearSVC com os melhores hiperparâmetros encontrados
svm_fast = LinearSVC(C=0.1, max_iter=5000)  # max_iter alto para garantir convergência

# Reduzir para 20.000 amostras para um teste rápido
X_teste, _, y_teste, _ = train_test_split(X_combinado, y_selecionado, train_size=20000, stratify=y_selecionado, random_state=42)

# Mensagem de início
print("🔄 Iniciando treinamento com LinearSVC...")
start_time = time.time()

# Treinar o modelo com 20.000 amostras
svm_fast.fit(X_teste, y_teste)

# Mensagem de fim
end_time = time.time()
tempo_total = end_time - start_time
print(f"✅ Treinamento finalizado em {tempo_total/60:.2f} minutos.")

🔄 Iniciando treinamento com LinearSVC...
✅ Treinamento finalizado em 0.84 minutos.


In [ ]:
# Criar e treinar o modelo com TODOS os dados
svm_final = LinearSVC(C=0.1, max_iter=5000)

# Mensagem de início
print("🔄 Iniciando treinamento com LinearSVC...")
start_time = time.time()

svm_final.fit(X_combinado, y_selecionado)

# Mensagem de fim
end_time = time.time()
tempo_total = end_time - start_time
print(f"✅ Treinamento finalizado em {tempo_total/60:.2f} minutos.")

🔄 Iniciando treinamento com LinearSVC...
✅ Treinamento finalizado em 3.44 minutos.


In [ ]:
# Fazer previsões
y_pred = svm_final.predict(X_test)

# Avaliar desempenho
print("🔹 Desempenho do modelo LinearSVC:")
print(classification_report(y_test, y_pred))

🔹 Desempenho do modelo LinearSVC:
              precision    recall  f1-score   support

           0       0.45      0.01      0.02      6022
           1       0.80      1.00      0.89     23978

    accuracy                           0.80     30000
   macro avg       0.63      0.50      0.46     30000
weighted avg       0.73      0.80      0.71     30000



In [ ]:
# Reduzir o dataset para 10.000 amostras para teste rápido
X_teste_menor, _, y_teste_menor, _ = train_test_split(X_combinado, y_selecionado, train_size=10000, stratify=y_selecionado, random_state=42)

# Testar SVM com kernel 'rbf'
print("🔄 Treinando SVM com kernel 'rbf'...")
start_time = time.time()
svm_rbf = SVC(kernel="rbf", C=0.1)
svm_rbf.fit(X_teste_menor, y_teste_menor)
end_time = time.time()
print(f"✅ Treino com 'rbf' finalizado em {(end_time - start_time)/60:.2f} minutos.")

# Fazer previsões
y_pred_rbf = svm_rbf.predict(X_test)

# Avaliar desempenho
print("🔹 Desempenho do SVM (kernel='rbf'):")
print(classification_report(y_test, y_pred_rbf))

# Testar SVM com kernel 'poly'
print("🔄 Treinando SVM com kernel 'poly'...")
start_time = time.time()
svm_poly = SVC(kernel="poly", C=0.1, degree=3)  # Grau 3 é um valor comum para 'poly'
svm_poly.fit(X_teste_menor, y_teste_menor)
end_time = time.time()
print(f"✅ Treino com 'poly' finalizado em {(end_time - start_time)/60:.2f} minutos.")

# Fazer previsões
y_pred_poly = svm_poly.predict(X_test)

# Avaliar desempenho
print("🔹 Desempenho do SVM (kernel='poly'):")
print(classification_report(y_test, y_pred_poly))


🔄 Treinando SVM com kernel 'rbf'...
✅ Treino com 'rbf' finalizado em 2.17 minutos.
🔹 Desempenho do SVM (kernel='rbf'):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      6022
           1       0.80      1.00      0.89     23978

    accuracy                           0.80     30000
   macro avg       0.40      0.50      0.44     30000
weighted avg       0.64      0.80      0.71     30000

🔄 Treinando SVM com kernel 'poly'...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


✅ Treino com 'poly' finalizado em 2.01 minutos.
🔹 Desempenho do SVM (kernel='poly'):
              precision    recall  f1-score   support

           0       0.44      0.01      0.02      6022
           1       0.80      1.00      0.89     23978

    accuracy                           0.80     30000
   macro avg       0.62      0.50      0.45     30000
weighted avg       0.73      0.80      0.71     30000



Balanceamento dos dados

In [ ]:
# Criar o modelo LinearSVC balanceado
svm_balanced = LinearSVC(C=0.1, max_iter=5000, class_weight='balanced')

print("🔄 Treinando LinearSVC balanceado...")
svm_balanced.fit(X_combinado, y_selecionado)

# Fazer previsões no conjunto de teste
y_pred_balanced = svm_balanced.predict(X_test)

# Avaliar desempenho
print("🔹 Desempenho do modelo LinearSVC balanceado:")
print(classification_report(y_test, y_pred_balanced))

🔄 Treinando LinearSVC balanceado...
🔹 Desempenho do modelo LinearSVC balanceado:
              precision    recall  f1-score   support

           0       0.29      0.61      0.39      6022
           1       0.86      0.63      0.73     23978

    accuracy                           0.63     30000
   macro avg       0.58      0.62      0.56     30000
weighted avg       0.75      0.63      0.66     30000



Novo teste, ajustando o valor de C

In [ ]:
# Definir os valores de C para testar
param_grid = {'C': [0.01, 0.1, 1, 10]}

# Configurar GridSearchCV para otimizar o F1-score médio (weighted)
grid_svm_balanced = GridSearchCV(
    LinearSVC(class_weight='balanced', max_iter=5000),
    param_grid,
    cv=3,
    scoring='f1_weighted',  # Maximizar F1-score ponderado
    verbose=3
)

print("🔄 Rodando GridSearch para encontrar o melhor C...")
grid_svm_balanced.fit(X_combinado, y_selecionado)

# Melhor configuração encontrada
best_C = grid_svm_balanced.best_params_['C']
print("🎯 Melhor parâmetro encontrado:", best_C)

# Treinar o modelo final com o melhor C
svm_final = LinearSVC(C=best_C, class_weight='balanced', max_iter=5000)
svm_final.fit(X_combinado, y_selecionado)

# Fazer previsões
y_pred_final = svm_final.predict(X_test)

# Avaliar desempenho final
from sklearn.metrics import classification_report
print("🔹 Desempenho do modelo otimizado:")
print(classification_report(y_test, y_pred_final))


🔄 Rodando GridSearch para encontrar o melhor C...
Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3] END ............................C=0.01;, score=0.660 total time= 1.6min
[CV 2/3] END ............................C=0.01;, score=0.642 total time= 1.1min
[CV 3/3] END ............................C=0.01;, score=0.660 total time= 1.1min
[CV 1/3] END .............................C=0.1;, score=0.657 total time= 3.0min
[CV 2/3] END .............................C=0.1;, score=0.643 total time= 3.5min
[CV 3/3] END .............................C=0.1;, score=0.659 total time= 2.7min
[CV 1/3] END ...............................C=1;, score=0.651 total time= 9.8min
[CV 2/3] END ...............................C=1;, score=0.639 total time=11.8min
[CV 3/3] END ...............................C=1;, score=0.664 total time=10.0min
[CV 1/3] END ..............................C=10;, score=0.639 total time=38.3min
[CV 2/3] END ..............................C=10;, score=0.628 total time=46.5min

In [ ]:
# Divisão dos dados (supondo que X e y já estejam prontos)
X_train, X_test, y_train, y_test = train_test_split(X_combinado, y_selecionado, test_size=0.5, random_state=42)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}")
print(f"Total de amostras: {X_train.shape[0] + X_test.shape[0]}")

# Modelo 1: LinearSVC sem balanceamento
modelo_puro = LinearSVC(C=0.01, dual=False, max_iter=10000)
modelo_puro.fit(X_train, y_train)
pred_puro = modelo_puro.predict(X_test)

# Modelo 2: LinearSVC com balanceamento
modelo_balanceado = LinearSVC(C=0.01, class_weight='balanced', dual=False, max_iter=10000)
modelo_balanceado.fit(X_train, y_train)
pred_balanceado = modelo_balanceado.predict(X_test)

# Exibir métricas de ambos os modelos
print("Desempenho do Modelo LinearSVC sem balanceamento:")
print(classification_report(y_test, pred_puro))

print("Desempenho do Modelo LinearSVC com balanceamento:")
print(classification_report(y_test, pred_balanceado))

Tamanho do conjunto de treino: 30000
Tamanho do conjunto de teste: 30000
Total de amostras: 60000
Desempenho do Modelo LinearSVC sem balanceamento:
              precision    recall  f1-score   support

           0       0.37      0.01      0.02      6022
           1       0.80      1.00      0.89     23978

    accuracy                           0.80     30000
   macro avg       0.58      0.50      0.45     30000
weighted avg       0.71      0.80      0.71     30000

Desempenho do Modelo LinearSVC com balanceamento:
              precision    recall  f1-score   support

           0       0.27      0.58      0.37      6022
           1       0.85      0.62      0.72     23978

    accuracy                           0.61     30000
   macro avg       0.56      0.60      0.54     30000
weighted avg       0.74      0.61      0.65     30000



In [ ]:
# Divisão dos dados (supondo que X e y já estejam prontos)
X_train, X_test, y_train, y_test = train_test_split(X_combinado, y, test_size=0.5, random_state=42)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}")
print(f"Total de amostras: {X_train.shape[0] + X_test.shape[0]}")

# Modelo 1: LinearSVC sem balanceamento
modelo_puro = LinearSVC(C=0.01, dual=False, max_iter=10000)
modelo_puro.fit(X_train, y_train)
pred_puro = modelo_puro.predict(X_test)

# Modelo 2: LinearSVC com balanceamento
modelo_balanceado = LinearSVC(C=0.01, class_weight='balanced', dual=False, max_iter=10000)
modelo_balanceado.fit(X_train, y_train)
pred_balanceado = modelo_balanceado.predict(X_test)

# Exibir métricas de ambos os modelos
print("Desempenho do Modelo LinearSVC sem balanceamento:")
print(classification_report(y_test, pred_puro))

print("Desempenho do Modelo LinearSVC com balanceamento:")
print(classification_report(y_test, pred_balanceado))

Tamanho do conjunto de treino: 58455
Tamanho do conjunto de teste: 58455
Total de amostras: 116910
Desempenho do Modelo LinearSVC sem balanceamento:
              precision    recall  f1-score   support

           0       0.70      0.26      0.38     10905
           1       0.85      0.97      0.91     47550

    accuracy                           0.84     58455
   macro avg       0.77      0.62      0.64     58455
weighted avg       0.82      0.84      0.81     58455

Desempenho do Modelo LinearSVC com balanceamento:
              precision    recall  f1-score   support

           0       0.45      0.73      0.56     10905
           1       0.93      0.79      0.86     47550

    accuracy                           0.78     58455
   macro avg       0.69      0.76      0.71     58455
weighted avg       0.84      0.78      0.80     58455



In [ ]:
# Divisão dos dados (supondo que X e y já estejam prontos)
X_train, X_test, y_train, y_test = train_test_split(X_combinado, y, test_size=0.3, random_state=42)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}")
print(f"Total de amostras: {X_train.shape[0] + X_test.shape[0]}")

# Modelo 1: LinearSVC sem balanceamento
modelo_puro = LinearSVC(C=0.01, dual=False, max_iter=10000)
modelo_puro.fit(X_train, y_train)
pred_puro = modelo_puro.predict(X_test)

# Modelo 2: LinearSVC com balanceamento
modelo_balanceado = LinearSVC(C=0.01, class_weight='balanced', dual=False, max_iter=10000)
modelo_balanceado.fit(X_train, y_train)
pred_balanceado = modelo_balanceado.predict(X_test)

# Exibir métricas de ambos os modelos
print("Desempenho do Modelo LinearSVC sem balanceamento:")
print(classification_report(y_test, pred_puro))

print("Desempenho do Modelo LinearSVC com balanceamento:")
print(classification_report(y_test, pred_balanceado))

Tamanho do conjunto de treino: 81837
Tamanho do conjunto de teste: 35073
Total de amostras: 116910
Desempenho do Modelo LinearSVC sem balanceamento:
              precision    recall  f1-score   support

           0       0.70      0.25      0.37      6543
           1       0.85      0.97      0.91     28530

    accuracy                           0.84     35073
   macro avg       0.77      0.61      0.64     35073
weighted avg       0.82      0.84      0.81     35073

Desempenho do Modelo LinearSVC com balanceamento:
              precision    recall  f1-score   support

           0       0.45      0.73      0.56      6543
           1       0.93      0.80      0.86     28530

    accuracy                           0.78     35073
   macro avg       0.69      0.76      0.71     35073
weighted avg       0.84      0.78      0.80     35073



Teste com o RBF

In [ ]:
import pickle

# Caminho do arquivo de checkpoint
CHECKPOINT_FILE = "checkpoint_svc.pkl"

# Função para salvar progresso
def salvar_checkpoint(modelo, nome_arquivo):
    with open(nome_arquivo, 'wb') as f:
        pickle.dump(modelo, f)

# Função para carregar checkpoint (se existir)
def carregar_checkpoint(nome_arquivo):
    if os.path.exists(nome_arquivo):
        with open(nome_arquivo, 'rb') as f:
            return pickle.load(f)
    return None

# Divisão dos dados (supondo que X e y já estejam prontos)
X_train, X_test, y_train, y_test = train_test_split(X_combinado, y, test_size=0.3, random_state=42)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}")
print(f"Total de amostras: {X_train.shape[0] + X_test.shape[0]}")

# Definir os modelos
modelos = {
    "SVC (RBF) sem balanceamento": SVC(C=0.01, kernel='rbf'),
    "SVC (RBF) com balanceamento": SVC(C=0.01, kernel='rbf', class_weight='balanced')
}

# Carregar progresso salvo, se existir
progresso_salvo = carregar_checkpoint(CHECKPOINT_FILE)
if progresso_salvo:
    print("\n🔄 Checkpoint encontrado! Retomando do último modelo treinado...\n")
    modelos = {k: v for k, v in modelos.items() if k not in progresso_salvo}

# Treinar os modelos com barra de progresso
resultados = progresso_salvo if progresso_salvo else {}
for nome, modelo in tqdm(modelos.items(), desc="Treinando modelos"):
    print(f"\n▶ Treinando {nome}...")
    modelo.fit(X_train, y_train)

    print(f"✅ {nome} treinado! Fazendo previsões...")
    pred = modelo.predict(X_test)

    # Salvar resultado
    resultados[nome] = classification_report(y_test, pred, output_dict=True)
    salvar_checkpoint(resultados, CHECKPOINT_FILE)

    print(f"📌 Resultados de {nome} salvos!\n")

# Exibir relatório final
print("\n📊 Relatório Final de Todos os Modelos:")
for nome, relatorio in resultados.items():
    print(f"\n🔹 {nome}")
    print(classification_report(y_test, pred))


Tamanho do conjunto de treino: 81837
Tamanho do conjunto de teste: 35073
Total de amostras: 116910


Treinando modelos:   0%|          | 0/2 [00:00<?, ?it/s]


▶ Treinando SVC (RBF) sem balanceamento...
✅ SVC (RBF) sem balanceamento treinado! Fazendo previsões...


Treinando modelos:  50%|█████     | 1/2 [2:24:02<2:24:02, 8642.51s/it]

📌 Resultados de SVC (RBF) sem balanceamento salvos!


▶ Treinando SVC (RBF) com balanceamento...
✅ SVC (RBF) com balanceamento treinado! Fazendo previsões...


Treinando modelos: 100%|██████████| 2/2 [7:47:55<00:00, 14037.57s/it]

📌 Resultados de SVC (RBF) com balanceamento salvos!


📊 Relatório Final de Todos os Modelos:

🔹 SVC (RBF) sem balanceamento
              precision    recall  f1-score   support

           0       0.44      0.69      0.53      6543
           1       0.92      0.80      0.85     28530

    accuracy                           0.78     35073
   macro avg       0.68      0.74      0.69     35073
weighted avg       0.83      0.78      0.79     35073


🔹 SVC (RBF) com balanceamento
              precision    recall  f1-score   support

           0       0.44      0.69      0.53      6543
           1       0.92      0.80      0.85     28530

    accuracy                           0.78     35073
   macro avg       0.68      0.74      0.69     35073
weighted avg       0.83      0.78      0.79     35073



Teste com o Poly

In [ ]:
import pickle

# Caminho do diretório para salvar checkpoints
CHECKPOINT_DIR = "/content/drive/My Drive/TCC/checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
CHECKPOINT_FILE = os.path.join(CHECKPOINT_DIR, "checkpoint_svc_poly.pkl")

# Função para salvar progresso
def salvar_checkpoint(modelo, nome_arquivo):
    with open(nome_arquivo, 'wb') as f:
        pickle.dump(modelo, f)

# Função para carregar checkpoint (se existir)
def carregar_checkpoint(nome_arquivo):
    if os.path.exists(nome_arquivo):
        with open(nome_arquivo, 'rb') as f:
            return pickle.load(f)
    return None

# Divisão dos dados (supondo que X e y já estejam prontos)
X_train, X_test, y_train, y_test = train_test_split(X_combinado, y, test_size=0.3, random_state=42)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}")
print(f"Total de amostras: {X_train.shape[0] + X_test.shape[0]}")

# Definir os modelos
modelos = {
    "SVC (Poly) sem balanceamento": SVC(C=0.01, kernel='poly'),
    "SVC (Poly) com balanceamento": SVC(C=0.01, kernel='poly', class_weight='balanced')
}

# Carregar progresso salvo, se existir
progresso_salvo = carregar_checkpoint(CHECKPOINT_FILE)
if progresso_salvo:
    print("\n🔄 Checkpoint encontrado! Retomando do último modelo treinado...\n")
    modelos = {k: v for k, v in modelos.items() if k not in progresso_salvo}

# Treinar os modelos com barra de progresso
resultados = progresso_salvo if progresso_salvo else {}
for nome, modelo in tqdm(modelos.items(), desc="Treinando modelos"):
    print(f"\n▶ Treinando {nome}...")
    modelo.fit(X_train, y_train)

    print(f"✅ {nome} treinado! Fazendo previsões...")
    pred = modelo.predict(X_test)

    # Salvar resultado
    resultados[nome] = classification_report(y_test, pred, output_dict=True)
    salvar_checkpoint(resultados, CHECKPOINT_FILE)

    print(f"📌 Resultados de {nome} salvos!")

# Exibir relatório final
print("\n📊 Relatório Final de Todos os Modelos:")
for nome, relatorio in resultados.items():
    print(f"\n🔹 {nome}")
    print(classification_report(y_test, pred))

Tamanho do conjunto de treino: 81837
Tamanho do conjunto de teste: 35073
Total de amostras: 116910

🔄 Checkpoint encontrado! Retomando do último modelo treinado...



Treinando modelos:   0%|          | 0/1 [00:00<?, ?it/s]


▶ Treinando SVC (Poly) com balanceamento...
✅ SVC (Poly) com balanceamento treinado! Fazendo previsões...


Treinando modelos: 100%|██████████| 1/1 [3:33:58<00:00, 12838.37s/it]

📌 Resultados de SVC (Poly) com balanceamento salvos!

📊 Relatório Final de Todos os Modelos:

🔹 SVC (Poly) sem balanceamento
              precision    recall  f1-score   support

           0       0.45      0.64      0.53      6543
           1       0.91      0.82      0.86     28530

    accuracy                           0.79     35073
   macro avg       0.68      0.73      0.70     35073
weighted avg       0.82      0.79      0.80     35073


🔹 SVC (Poly) com balanceamento
              precision    recall  f1-score   support

           0       0.45      0.64      0.53      6543
           1       0.91      0.82      0.86     28530

    accuracy                           0.79     35073
   macro avg       0.68      0.73      0.70     35073
weighted avg       0.82      0.79      0.80     35073



In [ ]:
import pickle

# Diretório para salvar checkpoints
diretorio_checkpoint = "/content/drive/My Drive/TCC/checkpoints"
os.makedirs(diretorio_checkpoint, exist_ok=True)
CHECKPOINT_FILE = os.path.join(diretorio_checkpoint, "checkpoint_svc_linear.pkl")

# Função para salvar progresso
def salvar_checkpoint(modelo, nome_arquivo):
    with open(nome_arquivo, 'wb') as f:
        pickle.dump(modelo, f)

# Função para carregar checkpoint (se existir)
def carregar_checkpoint(nome_arquivo):
    if os.path.exists(nome_arquivo):
        with open(nome_arquivo, 'rb') as f:
            return pickle.load(f)
    return None

# Divisão dos dados (supondo que X e y já estejam prontos)
X_train, X_test, y_train, y_test = train_test_split(X_combinado, y, test_size=0.3, random_state=42)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}")
print(f"Total de amostras: {X_train.shape[0] + X_test.shape[0]}")

# Definir o modelo
modelo_nome = "SVC (Linear)"
modelo = SVC(C=0.01, kernel='linear', class_weight='balanced')

# Carregar progresso salvo, se existir
progresso_salvo = carregar_checkpoint(CHECKPOINT_FILE)
if progresso_salvo:
    print("\n🔄 Checkpoint encontrado! Retomando do último progresso...")
    modelo = progresso_salvo
else:
    print("\n🚀 Iniciando o treinamento do modelo...")

# Treinar o modelo com barra de progresso
for _ in tqdm(range(1), desc=f"Treinando {modelo_nome}"):
    modelo.fit(X_train, y_train)
    salvar_checkpoint(modelo, CHECKPOINT_FILE)
    print(f"✅ {modelo_nome} treinado e checkpoint salvo!")

# Fazer previsões
print(f"\n▶ Fazendo previsões com {modelo_nome}...")
pred = modelo.predict(X_test)

# Exibir relatório final
print("\n📊 Relatório Final:")
print(classification_report(y_test, pred))


Tamanho do conjunto de treino: 81837
Tamanho do conjunto de teste: 35073
Total de amostras: 116910

🚀 Iniciando o treinamento do modelo...


Treinando SVC (Linear): 100%|██████████| 1/1 [2:48:59<00:00, 10139.60s/it]

✅ SVC (Linear) treinado e checkpoint salvo!

▶ Fazendo previsões com SVC (Linear)...



📊 Relatório Final:
              precision    recall  f1-score   support

           0       0.45      0.73      0.56      6543
           1       0.93      0.80      0.86     28530

    accuracy                           0.78     35073
   macro avg       0.69      0.76      0.71     35073
weighted avg       0.84      0.78      0.80     35073



In [ ]:
from sklearn.metrics import balanced_accuracy_score

#
# Divisão dos dados (supondo que X e y já estejam prontos)
X_train, X_test, y_train, y_test = train_test_split(X_combinado, y, test_size=0.3, random_state=42)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}")
print(f"Total de amostras: {X_train.shape[0] + X_test.shape[0]}")

# Modelo 1: LinearSVC sem balanceamento
modelo_puro = LinearSVC(C=0.01, dual=False, max_iter=10000)
modelo_puro.fit(X_train, y_train)
pred_puro = modelo_puro.predict(X_test)

# Modelo 2: LinearSVC com balanceamento
modelo_balanceado = LinearSVC(C=0.01, class_weight='balanced', dual=False, max_iter=10000)
modelo_balanceado.fit(X_train, y_train)
pred_balanceado = modelo_balanceado.predict(X_test)

# Exibir métricas de ambos os modelos
print("Desempenho do Modelo LinearSVC sem balanceamento:")
print(balanced_accuracy_score(y_test, pred_puro))

print("Desempenho do Modelo LinearSVC com balanceamento:")
print(balanced_accuracy_score(y_test, pred_balanceado))

Tamanho do conjunto de treino: 81837
Tamanho do conjunto de teste: 35073
Total de amostras: 116910
Desempenho do Modelo LinearSVC sem balanceamento:
0.6145930753650565
Desempenho do Modelo LinearSVC com balanceamento:
0.7611698586058451


# Treino do Naive Bayes

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, CategoricalNB
from sklearn.metrics import classification_report

# Diretório do checkpoint
CHECKPOINT_DIR = "/content/drive/My Drive/TCC/checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
CHECKPOINT_FILE = os.path.join(CHECKPOINT_DIR, "checkpoint_naive_bayes.pkl")

# Função para salvar progresso
def salvar_checkpoint(dados, nome_arquivo):
    with open(nome_arquivo, 'wb') as f:
        pickle.dump(dados, f)

# Função para carregar progresso
def carregar_checkpoint(nome_arquivo):
    if os.path.exists(nome_arquivo):
        with open(nome_arquivo, 'rb') as f:
            return pickle.load(f)
    return None

# Modelos a serem testados
modelos = {
    "GaussianNB": GaussianNB(),
    "BernoulliNB": BernoulliNB(),
}

# Carregar progresso salvo, se existir
progresso_salvo = carregar_checkpoint(CHECKPOINT_FILE)
if progresso_salvo:
    print("\n🔄 Checkpoint encontrado! Retomando do último modelo treinado...\n")
    modelos = {k: v for k, v in modelos.items() if k not in progresso_salvo}

# Armazenar resultados
resultados = progresso_salvo if progresso_salvo else {}

# Definir métricas de validação cruzada
metricas = ["accuracy", "precision_weighted", "recall_weighted", "f1_weighted"]

# Validar cada modelo com cross-validation
for nome, modelo in tqdm(modelos.items(), desc="Validando modelos Naive Bayes"):
    print(f"\n▶ Treinando e validando {nome}...")

    scores = cross_validate(modelo, X_train, y_train, cv=5, scoring=metricas)

    # Salvar métricas no dicionário de resultados
    resultados[nome] = {
        "accuracy": np.mean(scores["test_accuracy"]),
        "precision": np.mean(scores["test_precision_weighted"]),
        "recall": np.mean(scores["test_recall_weighted"]),
        "f1_score": np.mean(scores["test_f1_weighted"])
    }

    print(f"✅ {nome} validado!")
    print(f"📊 Acurácia: {resultados[nome]['accuracy']:.4f}")
    print(f"🎯 Precisão: {resultados[nome]['precision']:.4f}")
    print(f"🔁 Recall: {resultados[nome]['recall']:.4f}")
    print(f"📌 F1-score: {resultados[nome]['f1_score']:.4f}")

    # Salvar progresso
    salvar_checkpoint(resultados, CHECKPOINT_FILE)
    print(f"📌 Resultados de {nome} salvos!\n")

# Exibir relatório final
print("\n📊 Resultados da Validação Cruzada:")
for nome, dados in resultados.items():
    print(f"\n🔹 {nome}:")
    print(f"   ✅ Acurácia: {dados['accuracy']:.4f}")
    print(f"   🎯 Precisão: {dados['precision']:.4f}")
    print(f"   🔁 Recall: {dados['recall']:.4f}")
    print(f"   📌 F1-score: {dados['f1_score']:.4f}")

Validando modelos Naive Bayes:   0%|          | 0/2 [00:00<?, ?it/s]


▶ Treinando e validando GaussianNB...


Validando modelos Naive Bayes:  50%|█████     | 1/2 [00:14<00:14, 14.30s/it]

✅ GaussianNB validado!
📊 Acurácia: 0.5810
🎯 Precisão: 0.8002
🔁 Recall: 0.5810
📌 F1-score: 0.6259
📌 Resultados de GaussianNB salvos!


▶ Treinando e validando BernoulliNB...


Validando modelos Naive Bayes: 100%|██████████| 2/2 [00:31<00:00, 15.61s/it]

✅ BernoulliNB validado!
📊 Acurácia: 0.7148
🎯 Precisão: 0.8111
🔁 Recall: 0.7148
📌 F1-score: 0.7434
📌 Resultados de BernoulliNB salvos!


📊 Resultados da Validação Cruzada:

🔹 GaussianNB:
   ✅ Acurácia: 0.5810
   🎯 Precisão: 0.8002
   🔁 Recall: 0.5810
   📌 F1-score: 0.6259

🔹 BernoulliNB:
   ✅ Acurácia: 0.7148
   🎯 Precisão: 0.8111
   🔁 Recall: 0.7148
   📌 F1-score: 0.7434


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB, CategoricalNB
from sklearn.model_selection import cross_validate

# Caminho para salvar o checkpoint
CHECKPOINT_FILE = "/content/drive/My Drive/TCC/checkpoints/nb_multinomial_categorical.pkl"

# Função para salvar progresso
def salvar_checkpoint(modelo, nome_arquivo):
    with open(nome_arquivo, 'wb') as f:
        pickle.dump(modelo, f)

# Função para carregar checkpoint
def carregar_checkpoint(nome_arquivo):
    if os.path.exists(nome_arquivo):
        with open(nome_arquivo, 'rb') as f:
            return pickle.load(f)
    return {}

# Aplicar MinMaxScaler para garantir valores positivos
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definir os modelos a serem testados
modelos = {
    "MultinomialNB": MultinomialNB(),
    "CategoricalNB": CategoricalNB()
}

# Carregar progresso salvo, se existir
progresso_salvo = carregar_checkpoint(CHECKPOINT_FILE)
if progresso_salvo:
    print("\n🔄 Checkpoint encontrado! Retomando do último modelo validado...\n")
    modelos = {k: v for k, v in modelos.items() if k not in progresso_salvo}

# Validar os modelos com Cross-Validation
resultados = progresso_salvo if progresso_salvo else {}
metricas = ["accuracy", "precision_weighted", "recall_weighted", "f1_weighted"]

for nome, modelo in tqdm(modelos.items(), desc="Validando modelos Naive Bayes"):
    print(f"\n▶ Treinando e validando {nome}...")
    scores = cross_validate(modelo, X_train_scaled, y_train, cv=5, scoring=metricas)

    # Calcular médias das métricas
    resultados[nome] = {
        "accuracy": np.mean(scores.get("test_accuracy", [0])),
        "precision": np.mean(scores.get("test_precision_weighted", [0])),
        "recall": np.mean(scores.get("test_recall_weighted", [0])),
        "f1_score": np.mean(scores.get("test_f1_weighted", [0]))
    }

    print(f"✅ {nome} validado!")
    print(f"📊 Acurácia: {resultados[nome]['accuracy']:.4f}")
    print(f"🎯 Precisão: {resultados[nome]['precision']:.4f}")
    print(f"🔁 Recall: {resultados[nome]['recall']:.4f}")
    print(f"📌 F1-score: {resultados[nome]['f1_score']:.4f}")

    # Salvar progresso
    salvar_checkpoint(resultados, CHECKPOINT_FILE)
    print(f"📌 Resultados de {nome} salvos!\n")

# Exibir relatório final
top_model = max(resultados, key=lambda k: resultados[k]["f1_score"])
print("\n📊 Resultados Finais:")
for nome, metricas in resultados.items():
    print(f"\n🔹 {nome}:")
    print(f"   ✅ Acurácia: {metricas['accuracy']:.4f}")
    print(f"   🎯 Precisão: {metricas['precision']:.4f}")
    print(f"   🔁 Recall: {metricas['recall']:.4f}")
    print(f"   📌 F1-score: {metricas['f1_score']:.4f}")

print(f"\n🏆 Melhor modelo: {top_model} com F1-score de {resultados[top_model]['f1_score']:.4f}")

Validando modelos Naive Bayes:   0%|          | 0/2 [00:00<?, ?it/s]


▶ Treinando e validando MultinomialNB...


Validando modelos Naive Bayes:  50%|█████     | 1/2 [00:04<00:04,  4.98s/it]

✅ MultinomialNB validado!
📊 Acurácia: 0.8157
🎯 Precisão: 0.7983
🔁 Recall: 0.8157
📌 F1-score: 0.7345
📌 Resultados de MultinomialNB salvos!


▶ Treinando e validando CategoricalNB...


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 140, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 90, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_response.py", line 214, in _get_response_values
    y_pred = prediction_method(X)
             ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/naive_bayes.py", line 106, in predict
    j

✅ CategoricalNB validado!
📊 Acurácia: nan
🎯 Precisão: nan
🔁 Recall: nan
📌 F1-score: nan
📌 Resultados de CategoricalNB salvos!


📊 Resultados Finais:

🔹 MultinomialNB:
   ✅ Acurácia: 0.8157
   🎯 Precisão: 0.7983
   🔁 Recall: 0.8157
   📌 F1-score: 0.7345

🔹 CategoricalNB:
   ✅ Acurácia: nan
   🎯 Precisão: nan
   🔁 Recall: nan
   📌 F1-score: nan

🏆 Melhor modelo: MultinomialNB com F1-score de 0.7345


In [ ]:
CHECKPOINT_FILE = "checkpoint_multinomial.pkl"

def salvar_checkpoint(dados, nome_arquivo):
    with open(nome_arquivo, 'wb') as f:
        pickle.dump(dados, f)

def carregar_checkpoint(nome_arquivo):
    if os.path.exists(nome_arquivo):
        with open(nome_arquivo, 'rb') as f:
            return pickle.load(f)
    return None

# 🔹 Dividir os dados na proporção 70% treino / 30% teste
X_train, X_test, y_train, y_test = train_test_split(X_combinado, y, test_size=0.3, random_state=42, stratify=y)

# 🔹 Balanceamento
ros = RandomOverSampler(random_state=42)
X_train_bal, y_train_bal = ros.fit_resample(X_train, y_train)

print(f"\n📊 Antes do balanceamento: {Counter(y_train)}")
print(f"📊 Depois do balanceamento: {Counter(y_train_bal)}")

# 🔹 Normalização
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_bal_scaled = scaler.fit_transform(X_train_bal)

# 🔹 Modelo MultinomialNB
modelo = MultinomialNB()

# 🔹 Treino e teste (DADOS DESBALANCEADOS)
modelo.fit(X_train_scaled, y_train)
y_pred = modelo.predict(X_test_scaled)

# 🔹 Métricas
resultados = {
    "accuracy": accuracy_score(y_test, y_pred),
    "precision": precision_score(y_test, y_pred, average='weighted', zero_division=0),
    "recall": recall_score(y_test, y_pred, average='weighted', zero_division=0),
    "f1_score": f1_score(y_test, y_pred, average='weighted', zero_division=0)
}

print("\n🔹 MultinomialNB (DADOS DESBALANCEADOS):")
print(resultados)
print("\n", classification_report(y_test, y_pred, zero_division=0))

# 🔹 Treino e teste (DADOS BALANCEADOS)
modelo.fit(X_train_bal_scaled, y_train_bal)
y_pred_bal = modelo.predict(X_test_scaled)

# 🔹 Métricas
resultados_bal = {
    "accuracy": accuracy_score(y_test, y_pred_bal),
    "precision": precision_score(y_test, y_pred_bal, average='weighted', zero_division=0),
    "recall": recall_score(y_test, y_pred_bal, average='weighted', zero_division=0),
    "f1_score": f1_score(y_test, y_pred_bal, average='weighted', zero_division=0)
}

print("\n🔹 MultinomialNB (DADOS BALANCEADOS):")
print(resultados_bal)
print("\n", classification_report(y_test, y_pred_bal, zero_division=0))

# 🔹 Salvar resultados
salvar_checkpoint({"desbalanceado": resultados, "balanceado": resultados_bal}, CHECKPOINT_FILE)

# 🔹 Melhor modelo
melhor = max(resultados, key=lambda x: resultados[x])
print(f"\n🏆 Melhor abordagem: {melhor} com F1-score de {resultados[melhor]:.4f}")



📊 Antes do balanceamento: Counter({1: 66633, 0: 15204})
📊 Depois do balanceamento: Counter({1: 66633, 0: 66633})

🔹 MultinomialNB (DADOS DESBALANCEADOS):
{'accuracy': 0.814615231089442, 'precision': 0.7823774991925648, 'recall': 0.814615231089442, 'f1_score': 0.7327612392292684}

               precision    recall  f1-score   support

           0       0.64      0.00      0.01      6516
           1       0.81      1.00      0.90     28557

    accuracy                           0.81     35073
   macro avg       0.73      0.50      0.45     35073
weighted avg       0.78      0.81      0.73     35073


🔹 MultinomialNB (DADOS BALANCEADOS):
{'accuracy': 0.741596099563767, 'precision': 0.8190784943352176, 'recall': 0.741596099563767, 'f1_score': 0.7654320878535451}

               precision    recall  f1-score   support

           0       0.39      0.70      0.50      6516
           1       0.92      0.75      0.83     28557

    accuracy                           0.74     35073
   mac

In [ ]:
CHECKPOINT_FILE = "checkpoint_bernoulli_desbalanceado.pkl"

def salvar_checkpoint(dados, nome_arquivo):
    with open(nome_arquivo, 'wb') as f:
        pickle.dump(dados, f)

# 🔹 Dividir os dados na proporção 70% treino / 30% teste
X_train, X_test, y_train, y_test = train_test_split(X_combinado, y, test_size=0.3, random_state=42, stratify=y)

# 🔹 Normalização
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🔹 Modelo BernoulliNB
modelo = BernoulliNB()

# 🔹 Treinamento e teste com dados DESBALANCEADOS
modelo.fit(X_train_scaled, y_train)
y_pred = modelo.predict(X_test_scaled)

# 🔹 Métricas
resultados = {
    "accuracy": accuracy_score(y_test, y_pred),
    "precision": precision_score(y_test, y_pred, average='weighted', zero_division=0),
    "recall": recall_score(y_test, y_pred, average='weighted', zero_division=0),
    "f1_score": f1_score(y_test, y_pred, average='weighted', zero_division=0)
}

print("\n🔹 BernoulliNB (DADOS DESBALANCEADOS):")
print(resultados)
print("\n", classification_report(y_test, y_pred, zero_division=0))

# 🔹 Salvar resultados
salvar_checkpoint(resultados, CHECKPOINT_FILE)

# 🔹 Melhor métrica
melhor = max(resultados, key=lambda x: resultados[x])
print(f"\n🏆 Melhor métrica: {melhor} com F1-score de {resultados[melhor]:.4f}")



🔹 BernoulliNB (DADOS DESBALANCEADOS):
{'accuracy': 0.8124198101103413, 'precision': 0.7133921495809186, 'recall': 0.8124198101103413, 'f1_score': 0.7319152880941883}

               precision    recall  f1-score   support

           0       0.27      0.01      0.01      6516
           1       0.81      1.00      0.90     28557

    accuracy                           0.81     35073
   macro avg       0.54      0.50      0.45     35073
weighted avg       0.71      0.81      0.73     35073


🏆 Melhor métrica: accuracy com F1-score de 0.8124


In [ ]:
CHECKPOINT_FILE = "checkpoint_bernoulli_balanceado.pkl"

def salvar_checkpoint(dados, nome_arquivo):
    with open(nome_arquivo, 'wb') as f:
        pickle.dump(dados, f)

# 🔹 Dividir os dados na proporção 70% treino / 30% teste
X_train, X_test, y_train, y_test = train_test_split(X_combinado, y, test_size=0.3, random_state=42, stratify=y)

# 🔹 Balanceamento
ros = RandomOverSampler(random_state=42)
X_train_bal, y_train_bal = ros.fit_resample(X_train, y_train)

print(f"\n📊 Antes do balanceamento: {Counter(y_train)}")
print(f"📊 Depois do balanceamento: {Counter(y_train_bal)}")

# 🔹 Normalização
scaler = MinMaxScaler()
X_train_bal_scaled = scaler.fit_transform(X_train_bal)
X_test_scaled = scaler.transform(X_test)

# 🔹 Modelo BernoulliNB
modelo = BernoulliNB()

# 🔹 Treinamento e teste com dados BALANCEADOS
modelo.fit(X_train_bal_scaled, y_train_bal)
y_pred_bal = modelo.predict(X_test_scaled)

# 🔹 Métricas
resultados_bal = {
    "accuracy": accuracy_score(y_test, y_pred_bal),
    "precision": precision_score(y_test, y_pred_bal, average='weighted', zero_division=0),
    "recall": recall_score(y_test, y_pred_bal, average='weighted', zero_division=0),
    "f1_score": f1_score(y_test, y_pred_bal, average='weighted', zero_division=0)
}

print("\n🔹 BernoulliNB (DADOS BALANCEADOS):")
print(resultados_bal)
print("\n", classification_report(y_test, y_pred_bal, zero_division=0))

# 🔹 Salvar resultados
salvar_checkpoint(resultados_bal, CHECKPOINT_FILE)

# 🔹 Melhor métrica
melhor = max(resultados_bal, key=lambda x: resultados_bal[x])
print(f"\n🏆 Melhor métrica: {melhor} com F1-score de {resultados_bal[melhor]:.4f}")



📊 Antes do balanceamento: Counter({1: 66633, 0: 15204})
📊 Depois do balanceamento: Counter({1: 66633, 0: 66633})

🔹 BernoulliNB (DADOS BALANCEADOS):
{'accuracy': 0.1940238930231232, 'precision': 0.7789282121860173, 'recall': 0.1940238930231232, 'f1_score': 0.07640915221613886}

               precision    recall  f1-score   support

           0       0.19      1.00      0.31      6516
           1       0.91      0.01      0.02     28557

    accuracy                           0.19     35073
   macro avg       0.55      0.50      0.17     35073
weighted avg       0.78      0.19      0.08     35073


🏆 Melhor métrica: precision com F1-score de 0.7789


# Carregando o léxico e testando

In [ ]:
# Caminhos dos arquivos
path_csv = "/content/drive/MyDrive/TCC/OpLexicon-main/OpLexicon.csv"
path_txt = "/content/drive/MyDrive/TCC/OpLexicon-main/lexico_v3.0.txt"

# Verificar se os arquivos existem
print(f"Arquivo OpLexicon.csv {'encontrado' if os.path.exists(path_csv) else 'NÃO encontrado'}")
print(f"Arquivo lexico_v3.0.txt {'encontrado' if os.path.exists(path_txt) else 'NÃO encontrado'}")


Arquivo OpLexicon.csv encontrado
Arquivo lexico_v3.0.txt encontrado


In [ ]:
# Contar linhas do CSV
with open(path_csv, "r", encoding="utf-8") as f:
    num_lines_csv = sum(1 for line in f)

# Contar linhas do TXT
with open(path_txt, "r", encoding="utf-8") as f:
    num_lines_txt = sum(1 for line in f)

print(f"Linhas no OpLexicon.csv: {num_lines_csv}")
print(f"Linhas no lexico_v3.0.txt: {num_lines_txt}")

Linhas no OpLexicon.csv: 24474
Linhas no lexico_v3.0.txt: 32191


In [ ]:
# Mostrar as primeiras 10 linhas do CSV
print("\nPrimeiras linhas do OpLexicon.csv:")
with open(path_csv, "r", encoding="utf-8") as f:
    for _ in range(10):
        print(f.readline().strip())

# Mostrar as primeiras 10 linhas do TXT
print("\nPrimeiras linhas do lexico_v3.0.txt:")
with open(path_txt, "r", encoding="utf-8") as f:
    for _ in range(10):
        print(f.readline().strip())


Primeiras linhas do OpLexicon.csv:
abafada,adj,-1
abafadas,adj,-1
abafado,adj,-1
abafados,adj,-1
abafante,adj,-1
abafantes,adj,-1
abaixada,adj,-1
abaixadas,adj,-1
abaixado,adj,-1
abaixados,adj,-1

Primeiras linhas do lexico_v3.0.txt:
=[,emot,-1,A
=@,emot,-1,A
=p,emot,-1,A
=P,emot,-1,A
=x,emot,-1,A
=d,emot,1,A
=D,emot,1,A
;),emot,1,A
;),emot,1,A
;@,emot,-1,A


In [ ]:
import pandas as pd

# Carregar CSV
df_csv = pd.read_csv(path_csv, encoding="utf-8", sep=",", header=None)
df_csv.columns = ["palavra", "classe", "polaridade"]

# Carregar TXT (supondo que tenha a mesma estrutura)
df_txt = pd.read_csv(path_txt, encoding="utf-8", sep=",", header=None)
df_txt.columns = ["palavra", "classe", "polaridade", "tipo_polaridade"]

# Verificar o número total de palavras
print(f"Total de palavras no CSV: {df_csv.shape[0]}")
print(f"Total de palavras no TXT: {df_txt.shape[0]}")

# Verificar diferenças entre os arquivos
palavras_txt = set(df_txt["palavra"])
palavras_csv = set(df_csv["palavra"])

palavras_exclusivas_txt = palavras_txt - palavras_csv
print(f"Palavras que estão apenas no TXT: {len(palavras_exclusivas_txt)}")
print(list(palavras_exclusivas_txt)[:10])  # Mostrar apenas as 10 primeiras

# Palavras que mudaram de polaridade entre os arquivos

df_merged = df_csv.merge(df_txt, on="palavra", suffixes=("_csv", "_txt"))
diferenca_polaridade = df_merged[df_merged["polaridade_csv"] != df_merged["polaridade_txt"]]

print(f"Número de palavras com polaridade diferente: {diferenca_polaridade.shape[0]}")
print(diferenca_polaridade.head(10))  # Mostrar algumas diferenças

Total de palavras no CSV: 24474
Total de palavras no TXT: 32191
Palavras que estão apenas no TXT: 7673
['conformar', 'confederar-se', 'encrespar', 'descantar', 'empaçocar', 'excetuar', 'desvariar', 'alivelar', 'decodificar', 'desengranzar']
Número de palavras com polaridade diferente: 29
                palavra classe_csv  polaridade_csv classe_txt  polaridade_txt  \
2987           auxiliar        adj              -1         vb               0   
4893   cinematográficos        adj               0        adj               1   
4894   cinematográficos        adj               1        adj               0   
4916           circular        adj               1         vb               0   
6205              cruas        adj               0        adj               1   
6206              cruas        adj               1        adj               0   
6233            cubanos        adj               0        adj               1   
6234            cubanos        adj               1        adj  

In [ ]:
def get_polarity(word):
    result = df_txt[df_txt["palavra"] == word]
    if not result.empty:
        return result["polaridade"].values[0]
    return "Neutro/Não encontrado"

# Testando a função
print(get_polarity("feliz"))
print(get_polarity("chato"))
print(get_polarity("computador"))  # Palavra neutra


1
1
Neutro/Não encontrado


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_

# Integrando o Léxico

In [4]:
# Caminho para o arquivo do OpLexicon
LEXICON_PATH = "/content/drive/MyDrive/TCC/OpLexicon-main/lexico_v3.0.txt"

# Carregar o léxico separando os campos por vírgula
lexico_df = pd.read_csv(LEXICON_PATH, sep=",", header=None, names=["palavra", "classe", "polaridade", "origem"])

# Limpar a coluna de palavra (remover o símbolo '=')
lexico_df["palavra"] = lexico_df["palavra"].str.replace("=", "", regex=False)

# Visualizar
lexico_df.head()

,palavra,classe,polaridade,origem
0,[,emot,-1,A
1,@,emot,-1,A
2,p,emot,-1,A
3,P,emot,-1,A
4,x,emot,-1,A


In [5]:
# Criar dicionário {palavra: polaridade}
# A polaridade será: -1 (negativa), 1 (positiva), 0 (neutra)
lexico_dict = dict(zip(lexico_df["palavra"], lexico_df["polaridade"]))

Extrair pontuação de polaridade com base no léxico

In [13]:
import re

def calcular_polaridade_lexico(texto, lexico):
    palavras = re.findall(r'\b\w+\b', texto.lower())
    polaridade_total = sum([lexico.get(palavra, 0) for palavra in palavras])
    return polaridade_total

Carregando os dados originais

In [10]:
textos = []
y = []

# Pasta onde estão os arquivos JSON
pasta_dados = "/content/drive/MyDrive/TCC/Dados/Reviews Da Steam filtradas/Combined/part50/"

# Percorrer todos os arquivos JSON
for arquivo in tqdm(os.listdir(pasta_dados), desc="Lendo arquivos"):
    caminho = os.path.join(pasta_dados, arquivo)

    with open(caminho, "r", encoding="utf-8") as f:
        dados = json.load(f)

        # Garante que todas as chaves existem
        if "review" in dados and "voted_up" in dados:
            reviews_dict = dados["review"]
            votos_dict = dados["voted_up"]

            # Interseção para garantir que review e voted_up existem para o mesmo ID
            ids_validos = set(reviews_dict.keys()) & set(votos_dict.keys())

            for id_ in ids_validos:
                texto = reviews_dict[id_]
                rotulo = int(votos_dict[id_])  # True → 1, False → 0

                textos.append(texto)
                y.append(rotulo)

# Converter para NumPy array
y = np.array(y)

# Exibir resultado
print(f"Total de textos carregados: {len(textos)}")
print(f"Exemplo de texto: {textos[0][:100]}...")
print(f"Exemplo de rótulo: {y[0]}")

Lendo arquivos: 100%|██████████| 10/10 [00:02<00:00,  4.05it/s]

Total de textos carregados: 116910
Exemplo de texto: Bom jogo para passar o tempo, divertido e viciante....
Exemplo de rótulo: 1


In [6]:
textos = []

# Pasta onde estão os arquivos JSON
pasta_dados = "/content/drive/MyDrive/TCC/Dados/Reviews Da Steam filtradas/Combined/part50/"

# Percorrer todos os arquivos JSON
for arquivo in tqdm(os.listdir(pasta_dados), desc="Lendo arquivos"):
    caminho = os.path.join(pasta_dados, arquivo)

    with open(caminho, "r", encoding="utf-8") as f:
        dados = json.load(f)

        # Garante que todas as chaves existem
        if "review" in dados and "voted_up" in dados:
            reviews_dict = dados["review"]

            # Interseção para garantir que review e voted_up existem para o mesmo ID
            ids_validos = set(reviews_dict.keys())

            for id_ in ids_validos:
                texto = reviews_dict[id_]

                textos.append(texto)

# Exibir resultado
print(f"Total de textos carregados: {len(textos)}")
print(f"Exemplo de texto: {textos[0][:100]}...")

Lendo arquivos: 100%|██████████| 10/10 [00:11<00:00,  1.18s/it]

Total de textos carregados: 116910
Exemplo de texto: Péssimo....


Aplicando o léxico aos textos:

In [16]:
import re

#Esse código:
#Faz a tokenização simples dos textos.
#Soma as polaridades das palavras encontradas no léxico.
#Retorna um valor numérico por texto.

# Função para calcular a polaridade de um texto com base no léxico
def calcular_polaridade_lexico(texto, lexico):
    # Tokenização simples (palavras minúsculas sem pontuação)
    palavras = re.findall(r'\b\w+\b', texto.lower())

    polaridade_total = 0
    for palavra in palavras:
        if palavra in lexico:
            polaridade_total += lexico[palavra]

    return polaridade_total

# Aplicar a função a todos os textos
polaridades_lexico = [calcular_polaridade_lexico(texto, lexico_dict) for texto in textos]

# Exibir exemplo
for i in range(5):
    print(f"Texto {i+1} - Polaridade: {polaridades_lexico[i]}")

Texto 1 - Polaridade: 2
Texto 2 - Polaridade: 2
Texto 3 - Polaridade: 7
Texto 4 - Polaridade: 0
Texto 5 - Polaridade: 1


Separando em duas features (psoitivos e negativos)

In [17]:
# Converter a lista de textos para uma Series
textos_series = pd.Series(textos)

# Função que retorna duas features: número de palavras positivas e negativas
def calcular_polaridade_dupla(texto, lexico):
    palavras = re.findall(r'\b\w+\b', texto.lower())
    positivos = sum(1 for p in palavras if p in lexico and lexico[p] == 1)
    negativos = sum(1 for p in palavras if p in lexico and lexico[p] == -1)
    return pd.Series([positivos, negativos])

# Aplicar a função aos textos
X_lexico_df = textos_series.apply(lambda x: calcular_polaridade_dupla(x, lexico_dict))

# Renomear as colunas para deixar claro
X_lexico_df.columns = ['n_positivas', 'n_negativas']

# Visualizar resultado
print(X_lexico_df.head())

   n_positivas  n_negativas
0            2            0
1            2            0
2            7            0
3            0            0
4            1            0


Concatenar o léxico com embeddings

In [19]:
# Caminho para salvar o novo arquivo
caminho_saida = "/content/drive/MyDrive/TCC/Dados/X_completo_com_lexico.npy"

# Concatenar embeddings com as duas features do léxico
X_completo = np.concatenate([X_combinado, X_lexico_df.values], axis=1)

# Salvar o array concatenado
np.save(caminho_saida, X_completo)

print(f"Arquivo salvo com sucesso em: {caminho_saida}")
print(f"Formato do array salvo: {X_completo.shape}")

Arquivo salvo com sucesso em: /content/drive/MyDrive/TCC/Dados/X_completo_com_lexico.npy
Formato do array salvo: (116910, 2002)


Carregando o arquivo completo

In [3]:
X_completo = np.load("/content/drive/MyDrive/TCC/Dados/X_completo_com_lexico.npy")

Salvar o Y

In [21]:
# Salvar os rótulos no formato .npy
np.save("/content/drive/MyDrive/TCC/Dados/y.npy", y)

print("Rótulos salvos com sucesso!")

Rótulos salvos com sucesso!


Carregar o Y

In [4]:
y = np.load("/content/drive/MyDrive/TCC/Dados/y.npy")

# Separar dado de teste e dado de treino com o Léxico

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X_completo, y, test_size=0.2, random_state=42, stratify=y
)

# Treino e teste SVM

In [8]:
# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_completo, y, test_size=0.4, random_state=42)

# Verificar tamanhos
print(f"Tamanho do conjunto de treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}")
print(f"Total de amostras: {X_train.shape[0] + X_test.shape[0]}")

# Modelo 1: LinearSVC sem balanceamento
modelo_lexico = LinearSVC(C=0.01, dual=False, max_iter=10000)
modelo_lexico.fit(X_train, y_train)
pred_lexico = modelo_lexico.predict(X_test)

# Modelo 2: LinearSVC com balanceamento
modelo_lexico_bal = LinearSVC(C=0.01, class_weight='balanced', dual=False, max_iter=10000)
modelo_lexico_bal.fit(X_train, y_train)
pred_lexico_bal = modelo_lexico_bal.predict(X_test)

# Exibir métricas de ambos os modelos
print("🔸 Desempenho do Modelo LinearSVC com léxico (sem balanceamento):")
print(classification_report(y_test, pred_lexico))

print("🔸 Desempenho do Modelo LinearSVC com léxico (com balanceamento):")
print(classification_report(y_test, pred_lexico_bal))

Tamanho do conjunto de treino: 70146
Tamanho do conjunto de teste: 46764
Total de amostras: 116910
🔸 Desempenho do Modelo LinearSVC com léxico (sem balanceamento):
              precision    recall  f1-score   support

           0       0.33      0.00      0.00      8695
           1       0.81      1.00      0.90     38069

    accuracy                           0.81     46764
   macro avg       0.57      0.50      0.45     46764
weighted avg       0.72      0.81      0.73     46764

🔸 Desempenho do Modelo LinearSVC com léxico (com balanceamento):
              precision    recall  f1-score   support

           0       0.25      0.63      0.36      8695
           1       0.87      0.57      0.69     38069

    accuracy                           0.58     46764
   macro avg       0.56      0.60      0.52     46764
weighted avg       0.76      0.58      0.63     46764



In [22]:
!pip install -U imbalanced-learn

In [24]:
from imblearn.over_sampling import SMOTE

# Dividir os dados em treino e teste (60/40, como antes)
X_train, X_test, y_train, y_test = train_test_split(
    X_completo, y, test_size=0.4, random_state=42
)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}")
print(f"Total de amostras: {X_train.shape[0] + X_test.shape[0]}")

# Aplicar SMOTE APENAS no conjunto de treino
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print(f"\n🔄 Após SMOTE:")
print(f"Amostras de treino após balanceamento: {X_train_res.shape[0]}")
print(f"Distribuição das classes: {np.bincount(y_train_res)}")

# Treinar o modelo com os dados balanceados
modelo_smote = LinearSVC(C=0.01, dual=False, max_iter=10000)
modelo_smote.fit(X_train_res, y_train_res)

# Avaliar no conjunto de teste original
pred_smote = modelo_smote.predict(X_test)

print("\n🔸 Desempenho do Modelo LinearSVC com SMOTE:")
print(classification_report(y_test, pred_smote))


Tamanho do conjunto de treino: 70146
Tamanho do conjunto de teste: 46764
Total de amostras: 116910

🔄 Após SMOTE:
Amostras de treino após balanceamento: 114258
Distribuição das classes: [57129 57129]

🔸 Desempenho do Modelo LinearSVC com SMOTE:
              precision    recall  f1-score   support

           0       0.19      0.50      0.28      8703
           1       0.82      0.52      0.63     38061

    accuracy                           0.51     46764
   macro avg       0.51      0.51      0.46     46764
weighted avg       0.70      0.51      0.57     46764



In [5]:
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import joblib

# 🔹 Diretório para salvar os checkpoints
checkpoint_dir = "/content/drive/My Drive/TCC/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# 🔹 Divisão dos dados
X_train, X_test, y_train, y_test = train_test_split(
    X_completo, y, test_size=0.4, random_state=42
)
print(f"Tamanho do conjunto de treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]}")
print(f"Total de amostras: {X_train.shape[0] + X_test.shape[0]}")

# 🔹 Pipeline com oversampling
pipeline = ImbPipeline([
    ("scaler", StandardScaler()),
    ("smote", SMOTE(random_state=42)),
    ("clf", LinearSVC(dual=False, max_iter=10000))
])

# 🔹 Parâmetros para ajustar
param_grid = {
    "clf__C": [0.001, 0.01, 0.1, 1]
}

# 🔹 GridSearch com validação cruzada
grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=3,
    verbose=1,
    n_jobs=-1
)

# 🔹 Ajuste com barra de progresso
print("🔁 Iniciando GridSearch com SMOTE...\n")
grid.fit(X_train, y_train)

# 🔹 Melhor modelo
melhor_modelo = grid.best_estimator_
melhor_C = grid.best_params_["clf__C"]
print(f"\n✅ Melhor valor de C: {melhor_C}")

# 🔹 Previsão e relatório
y_pred = melhor_modelo.predict(X_test)
print("\n🔸 Desempenho do Modelo LinearSVC com SMOTE e melhor C:")
print(classification_report(y_test, y_pred))

# 🔹 Salvar checkpoint do modelo treinado
checkpoint_path = os.path.join(checkpoint_dir, f"modelo_SVM_C{melhor_C}.joblib")
joblib.dump(melhor_modelo, checkpoint_path)
print(f"\n💾 Modelo salvo em: {checkpoint_path}")

Tamanho do conjunto de treino: 70146
Tamanho do conjunto de teste: 46764
Total de amostras: 116910
🔁 Iniciando GridSearch com SMOTE...

Fitting 3 folds for each of 2 candidates, totalling 6 fits


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}